## Youtube 2 Mp3
The aim of this notebook is to test out some functions regarding automatic youtube video downloading into mp3 files, as well as moving those files into external drives

In [1]:
import os
import subprocess
import time
import flask
import requests
import pytube

### Download the video

In [10]:
import sys

def progress_function(chunk, file_handle, bytes_remaining):
    global filesize
    current = ((filesize - bytes_remaining)/filesize)
    percent = ('{0:.1f}').format(current*100)
    progress = int(50*current)
    status = '█' * progress + '-' * (50 - progress)
    sys.stdout.write(' ↳ |{bar}| {percent}%\r'.format(bar=status, percent=percent))
    sys.stdout.flush()

def download_youtube_video_as_mp3(url):
  # create a directory called 'media' if it doesn't exist
  if not os.path.exists('media'):
    os.makedirs('media')
  
  # download the youtube video
  youtube = pytube.YouTube(url)
  video = youtube.streams.filter(only_audio=True).first()
  video_size = video.filesize
  video_path = os.path.join('media', video.default_filename)
  video.on_success = progress_function
  print('Downloading: {}'.format(video.default_filename))
  video.download('media/', filename=video.default_filename)
  
  # convert the video to mp3
  mp3_path = os.path.join('media', video.default_filename.replace('.mp4', '.mp3'))
  subprocess.run(['ffmpeg', '-i', video_path, mp3_path])
  
  # delete the original video file
  os.remove(video_path)

  # print the downloaded confirmatio
  print('Downloaded: {}'.format(video.default_filename))

  # return the file name of the mp3
  return os.path.basename(mp3_path)


In [3]:
# download_youtube_video_as_mp3('https://www.youtube.com/watch?v=NmVrnZAAklU&t=1714s')
# this works great!

### Move it to the external drive

In [4]:
import shutil

def move_file_to_external_drive(file_name):
    # get the full path of the file
    file_path = os.path.join('media', file_name)
    
    # cut the file name from the path
    file_name = os.path.basename(file_path)
    
    # define the destination path in the external drive
    external_drive_path = 'D:'
    if os.path.exists(external_drive_path):
        destination_path = os.path.join(external_drive_path, file_name)
        # move the file to the external drive
        shutil.move(file_path, destination_path)
        print(f"File '{file_name}' moved to external drive.")
    else:
        print(f"External drive not found at '{external_drive_path}'. File '{file_name}' not moved.")


In [5]:
# move_file_to_external_drive('Never Gonna Give You Up.mp3')
# it works!

# Download and move to external drive

In [12]:
def download_and_store_mp3(url=None):
    try:
        if url is None:
            url = input("Enter the YouTube video URL: ")
        mp3_file_name = download_youtube_video_as_mp3(url)
        move_file_to_external_drive(mp3_file_name)
        return mp3_file_name
    except pytube.exceptions.RegexMatchError:
        print("Invalid YouTube video URL. Please enter a valid URL.")
    except pytube.exceptions.VideoUnavailable:
        print("The requested video is unavailable. Please try again later.")
    except pytube.exceptions.ExtractError:
        print("An error occurred while extracting the video. Please try again later.")
    except pytube.exceptions.VideoPrivate:
        print("The requested video is private. Please provide a video that is not private.")
    except Exception as e:
        print(f"An error occurred: {e}")


In [13]:
download_and_store_mp3('https://www.youtube.com/watch?v=dQw4w9WgXcQ')
#  okay its working

Downloading: Never Gonna Give You Up.mp4
Downloaded: Never Gonna Give You Up.mp4
File 'Never Gonna Give You Up.mp3' moved to external drive.


'Never Gonna Give You Up.mp3'

In [9]:
# download_and_store_mp3()

## Batch inputs
Takes a list of youtube links and downloads them into mp3 files, then moves them to the external drive

In [16]:
def download_and_store_mp3_batch(urls):
    total_files = len(urls)
    downloaded_files = 0
    for url in urls:
        try:
            mp3_file_name = download_and_store_mp3(url)
            downloaded_files += 1
            print(f"({downloaded_files}/{total_files} files downloaded)")
        except pytube.exceptions.RegexMatchError:
            print(f"Invalid YouTube video URL '{url}'. Skipping...")
        except pytube.exceptions.VideoUnavailable:
            print(f"The requested video '{url}' is unavailable. Skipping...")
        except pytube.exceptions.ExtractError:
            print(f"An error occurred while extracting the video '{url}'. Skipping...")
        except pytube.exceptions.VideoPrivate:
            print(f"The requested video '{url}' is private. Skipping...")
        except Exception as e:
            print(f"An error occurred while processing the video '{url}': {e}. Skipping...")


In [17]:
# urls = ['https://www.youtube.com/watch?v=dQw4w9WgXcQ', 'https://www.youtube.com/watch?v=ZHwVBirqD2s']
# download_and_store_mp3_batch(urls)
# it works!

Downloading: Never Gonna Give You Up.mp4
Downloaded: Never Gonna Give You Up.mp4
File 'Never Gonna Give You Up.mp3' moved to external drive.
(1/2 files downloaded)
Downloading: Im Still Standing.mp4
Downloaded: Im Still Standing.mp4
File 'Im Still Standing.mp3' moved to external drive.
(2/2 files downloaded)


In [23]:
import ipywidgets as widgets
from IPython.display import display

def download_and_store_mp3_ui():
    # create a text area for the user to input the URLs
    urls_textarea = widgets.Textarea(
        placeholder='Enter YouTube video URLs, one per line',
        description='YouTube URLs:',
        layout=widgets.Layout(width='100%', height='200px')
    )
    
    # create a button to start the download process
    download_button = widgets.Button(description='Download and store mp3 files')
    
    # create an output area to display the progress and results
    output_area = widgets.Output()
    
    # define the function to be called when the download button is clicked
    def on_download_button_clicked(b):
        # get the URLs from the text area
        urls = urls_textarea.value.split('\n').filter(lambda url: url)
        
        # download and store the mp3 files
        download_and_store_mp3_batch(urls)
    
    # attach the on_download_button_clicked function to the download button
    download_button.on_click(on_download_button_clicked)
    
    # display the UI
    display(widgets.VBox([urls_textarea, download_button, output_area]))


In [24]:
download_and_store_mp3_ui()

AttributeError: 'list' object has no attribute 'filter'